In [32]:
import pandas as pd
import seaborn as sns
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

df = pd.read_csv("train.csv")
display(df)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [33]:
#전처리
df_selected = df[{"Survived", "Pclass", "Sex", "Age", "Parch", "SibSp", "Fare", "Embarked"}].dropna()

df_selected["Sex_number"] = df_selected["Sex"].replace("female", 1).replace("male", 2)
df_selected["Embarked_number"] = df_selected["Embarked"].replace("C", 1).replace("Q", 2).replace("S", 3)
display(df_selected)
# sns.pairplot(df_selected)

,Fare,Sex,SibSp,Parch,Embarked,Age,Survived,Pclass,Sex_number,Embarked_number
0,7.2500,male,1,0,S,22.0,0,3,2,3
1,71.2833,female,1,0,C,38.0,1,1,1,1
2,7.9250,female,0,0,S,26.0,1,3,1,3
3,53.1000,female,1,0,S,35.0,1,1,1,3
4,8.0500,male,0,0,S,35.0,0,3,2,3
...,...,...,...,...,...,...,...,...,...,...
885,29.1250,female,0,5,Q,39.0,0,3,1,2
886,13.0000,male,0,0,S,27.0,0,2,2,3
887,30.0000,female,0,0,S,19.0,1,1,1,3
889,30.0000,male,0,0,C,26.0,1,1,2,1


In [37]:
#데이터셋 분리
x = df_selected[{"Pclass", "Sex_number", "Age", "Parch", "SibSp", "Fare", "Embarked_number"}]
y = df_selected["Survived"]
# display(x)
# display(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=True, random_state=34)

print(x_train, y_train)

     Embarked_number      Fare  SibSp  Parch   Age  Sex_number  Pclass
114                1   14.4583      0      0  17.0           1       3
637                3   26.2500      1      1  31.0           2       2
759                3   86.5000      0      0  33.0           1       1
258                1  512.3292      0      0  35.0           1       1
443                3   13.0000      0      0  28.0           1       2
..               ...       ...    ...    ...   ...         ...     ...
403                3   15.8500      1      0  28.0           2       3
111                1   14.4542      1      0  14.5           1       3
622                1   15.7417      1      1  20.0           2       3
152                3    8.0500      0      0  55.5           2       3
526                3   10.5000      0      0  50.0           1       2

[569 rows x 7 columns] 114    0
637    0
759    1
258    1
443    1
      ..
403    0
111    0
622    1
152    0
526    1
Name: Survived, Length: 5

In [38]:
ml = SVC(kernel="linear", C=1.0, random_state=0)
ml.fit(x_train, y_train)
y_pred = ml.predict(x_test)
print("SVM 총 테스트 갯수 : %d, 오류 갯수 : %d" %(len(y_test), (y_test != y_pred).sum()))
print("SVM 정확도 %.2f" %accuracy_score(y_test, y_pred))

SVM 총 테스트 갯수 : 143, 오류 갯수 : 32
SVM 정확도 0.78


In [39]:
clf = LogisticRegression(solver='lbfgs').fit(x_train,y_train)
y_pred = clf.predict(x_test)
print("Logistic 총 테스트 갯수 : %d, 오류 갯수 : %d" %(len(y_test), (y_test != y_pred).sum()))
print("Logistic 정확도 %.2f" %accuracy_score(y_test, y_pred))

Logistic 총 테스트 갯수 : 143, 오류 갯수 : 28
Logistic 정확도 0.80


/Users/hrdkdh/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
